In [4]:
video_frame = """
<script>
hello
</script>
"""

# convert video_frame to bytes
video_frame_bytes = bytes(video_frame.encode())

# display eval_js using video_frame_bytes
display(Javascript(eval_js(video_frame_bytes)))

TypeError: b'\n<script>\nhello\n</script>\n' is not JSON serializable

In [ ]:
import IPython
print(IPython.__version__)

In [12]:
# import necessary packages
import cv2
import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
import torch

# load the model
model = torch.load('/content/best.pt')

# function to perform real-time object detection
def detect():

    # define the video frame JS code
    video_frame = """
    <script>
      var video = document.querySelector("#myVideoElement");

      // resize the canvas to match the video dimensions
      document.querySelector("#myCanvasElement").width = video.width;
      document.querySelector("#myCanvasElement").height = video.height;

      // get canvas context for drawing
      var canvas = document.querySelector("#myCanvasElement");
      var ctx = canvas.getContext('2d');

      // draw the video to the canvas every 16ms (60fps)
      setInterval(() => {
        ctx.drawImage(video, 0, 0);
      }, 16);

      // data URL for the current canvas image
      function getCanvasImage() {
        return canvas.toDataURL('image/jpeg', 0.8);
      }
    </script>

    <video id="myVideoElement" controls autoplay></video>
    <canvas id="myCanvasElement" style="display: none"></canvas>
    """

    # display js in the output to prevent scrolling (or you can scroll forever)
    display(Javascript(video_frame))


    # convert video_frame to bytes
    video_frame_bytes = bytes(video_frame.encode())

    # display eval_js using video_frame_bytes
    display(Javascript(eval_js(video_frame_bytes)))

    # access user's webcam and capture image for real-time detection
    js = Javascript('''
      async function useCam() {
        // use navigator.mediaDevices.getUserMedia to obtain camera stream
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        // display the stream as a video element
        const video = document.querySelector("#myVideoElement");
        video.srcObject = stream;

        // wait for video to play, then call window.saveFrame
        video.onloadedmetadata = (e) => {
          video.play();
          // the callback function that gets the current frame from the displayed video
          window.saveFrame = async () => {
            // create a canvas and draw the current video frame to it
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const ctx = canvas.getContext('2d');
            ctx.drawImage(video, 0, 0, canvas.width, canvas.height);
            // get the image data URL and convert to a base64-encoded byte array
            const imageDataURL = canvas.toDataURL('image/jpeg', 0.8);
            const imageData = imageDataURL.replace(/^data:image\/(png|jpeg);base64,/, "");
            // call window.py_save_frame in Python kernel with the image byte array as argument
            google.colab.kernel.invokeFunction('py_save_frame', [imageData], {});
          }
        }
      }
    ''')

In [13]:
detect()

<IPython.core.display.Javascript object>

TypeError: b'\n    <script>\n      var video = document.querySelector("#myVideoElement");\n\n      // resize the canvas to match the video dimensions\n      document.querySelector("#myCanvasElement").width = video.width;\n      document.querySelector("#myCanvasElement").height = video.height;\n\n      // get canvas context for drawing\n      var canvas = document.querySelector("#myCanvasElement");\n      var ctx = canvas.getContext(\'2d\');\n\n      // draw the video to the canvas every 16ms (60fps)\n      setInterval(() => {\n        ctx.drawImage(video, 0, 0);\n      }, 16);\n\n      // data URL for the current canvas image\n      function getCanvasImage() {\n        return canvas.toDataURL(\'image/jpeg\', 0.8);\n      }\n    </script>\n\n    <video id="myVideoElement" controls autoplay></video>\n    <canvas id="myCanvasElement" style="display: none"></canvas>\n    ' is not JSON serializable